In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import re
from pathlib import Path
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [2]:
# pip install psycopg2

In [3]:
file = Path('Resources/Airline_Delay_Cause.csv')

df = pd.read_csv(file) 

In [4]:
df

,year_,month_,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2023,3,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",89.0,8.0,4.46,1.00,...,0.00,0.93,1.0,1.0,412.0,262.0,38.0,53.0,0.0,59.0
1,2023,3,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",62.0,8.0,3.95,0.37,...,0.00,2.40,0.0,0.0,357.0,188.0,7.0,44.0,0.0,118.0
2,2023,3,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",11.0,2.0,1.00,0.00,...,0.00,1.00,0.0,0.0,60.0,24.0,0.0,0.0,0.0,36.0
3,2023,3,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",201.0,27.0,13.04,0.46,...,0.00,6.44,7.0,1.0,1336.0,742.0,13.0,220.0,0.0,361.0
4,2023,3,9E,Endeavor Air Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",1598.0,222.0,57.22,8.08,...,0.00,95.38,8.0,6.0,18248.0,7265.0,774.0,3458.0,0.0,6751.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21579,2022,3,YX,Republic Airline,SYR,"Syracuse, NY: Syracuse Hancock International",38.0,10.0,4.05,0.00,...,0.00,4.19,1.0,0.0,1089.0,249.0,0.0,83.0,0.0,757.0
21580,2022,3,YX,Republic Airline,TPA,"Tampa, FL: Tampa International",97.0,19.0,3.19,2.39,...,0.87,9.16,4.0,0.0,1197.0,306.0,181.0,131.0,13.0,566.0
21581,2022,3,YX,Republic Airline,TUL,"Tulsa, OK: Tulsa International",8.0,1.0,0.00,0.00,...,0.00,0.99,0.0,0.0,214.0,0.0,1.0,2.0,0.0,211.0
21582,2022,3,YX,Republic Airline,VPS,"Valparaiso, FL: Eglin AFB Destin Fort Walton B...",25.0,8.0,2.28,0.00,...,0.00,1.06,2.0,0.0,369.0,143.0,0.0,161.0,0.0,65.0


In [5]:
# Review columns in data
df.columns

Index(['year_', 'month_', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [6]:
# Check data types
df.dtypes

year_                    int64
month_                   int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
dtype: object

In [7]:
# Review unique carrier names to identify what will be excluded
df.carrier_name.unique()

array(['Endeavor Air Inc.', 'American Airlines Inc.',
       'Alaska Airlines Inc.', 'JetBlue Airways', 'Delta Air Lines Inc.',
       'Frontier Airlines Inc.', 'Allegiant Air',
       'Hawaiian Airlines Inc.', 'Envoy Air', 'Spirit Air Lines',
       'PSA Airlines Inc.', 'SkyWest Airlines Inc.',
       'United Air Lines Inc.', 'Southwest Airlines Co.',
       'Republic Airline', 'Horizon Air', 'Mesa Airlines Inc.'],
      dtype=object)

In [8]:
# Remove airlines that will not be used in our project
# df = df.loc[(df["carrier_name"] != "Alaska Airlines Inc.")]
# df = df.loc[(df["carrier_name"] != "Endeavor Air Inc.")]
# df = df.loc[(df["carrier_name"] != "Envoy Air")]
# df = df.loc[(df["carrier_name"] != "Hawaiian Airlines Inc.")]
# df = df.loc[(df["carrier_name"] != "Mesa Airlines Inc.")]
# df = df.loc[(df["carrier_name"] != "PSA Airlines Inc.")]
# df = df.loc[(df["carrier_name"] != "Republic Airline")]
# df = df.loc[(df["carrier_name"] != "Horizon Air")]
# df

In [9]:
# Confirm that airlines were removed
df.carrier_name.unique()

array(['Endeavor Air Inc.', 'American Airlines Inc.',
       'Alaska Airlines Inc.', 'JetBlue Airways', 'Delta Air Lines Inc.',
       'Frontier Airlines Inc.', 'Allegiant Air',
       'Hawaiian Airlines Inc.', 'Envoy Air', 'Spirit Air Lines',
       'PSA Airlines Inc.', 'SkyWest Airlines Inc.',
       'United Air Lines Inc.', 'Southwest Airlines Co.',
       'Republic Airline', 'Horizon Air', 'Mesa Airlines Inc.'],
      dtype=object)

In [10]:
# Create a new column that calculates delayed % of flights
df["arr_del15_per"] = df["arr_del15"] / df["arr_flights"]

# Create a new column that calculates delayed flights due to weather
df["arr_weather_delay_per"] = df["weather_ct"] / df["arr_flights"]

# Create a new column that calculates delayed flights due to air carrier
df["arr_carrier_delay_per"] = df["carrier_ct"] / df["arr_flights"]

# Create a new column that calculates delayed flights due to National Aviation System (e.g. heavy air traffic)
df["arr_nas_delay_per"] = df["nas_ct"] / df["arr_flights"]

# Create a new column that calculates delayed flights as a result of another flight on same aircraft delayed
df["arr_late_aircraft_delay_per"] = df["late_aircraft_ct"] / df["arr_flights"]




# Create a new column that calculates cancelled % of flights
df["arr_cancelled_per"] = df["arr_cancelled"] / df["arr_flights"]

# Create a new column that calculates cancelled flights due to security breach
df["arr_security_cancelled_per"] = df["security_ct"] / df["arr_flights"]




# Create a new column that calculates average delay time
# df["avg_delay_mins"] = df["carrier_delay"].mean



In [11]:
# Split airport_name column into 2 separate columns to identify city and state
df[['location', 'airport']] = df['airport_name'].str.split(pat=':', expand = True)
df[['location', 'airport']]

,location,airport
0,"Albany, GA",Southwest Georgia Regional
1,"Alexandria, LA",Alexandria International
2,"Augusta, GA",Augusta Regional at Bush Field
3,"Albany, NY",Albany International
4,"Atlanta, GA",Hartsfield-Jackson Atlanta International
...,...,...
21579,"Syracuse, NY",Syracuse Hancock International
21580,"Tampa, FL",Tampa International
21581,"Tulsa, OK",Tulsa International
21582,"Valparaiso, FL",Eglin AFB Destin Fort Walton Beach


In [12]:
# Delete airport_name column as it's no longer needed and now split
del df['airport_name']
df

,year_,month_,carrier,carrier_name,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,...,security_delay,late_aircraft_delay,arr_del15_per,arr_weather_delay_per,arr_carrier_delay_per,arr_nas_delay_per,arr_late_aircraft_delay_per,arr_cancelled_per,arr_security_cancelled_per,location
0,2023,3,9E,Endeavor Air Inc.,Southwest Georgia Regional,89.0,8.0,4.46,1.00,1.61,...,0.0,59.0,0.089888,0.011236,0.050112,0.018090,0.010449,0.011236,0.000000,"Albany, GA"
1,2023,3,9E,Endeavor Air Inc.,Alexandria International,62.0,8.0,3.95,0.37,1.29,...,0.0,118.0,0.129032,0.005968,0.063710,0.020806,0.038710,0.000000,0.000000,"Alexandria, LA"
2,2023,3,9E,Endeavor Air Inc.,Augusta Regional at Bush Field,11.0,2.0,1.00,0.00,0.00,...,0.0,36.0,0.181818,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,"Augusta, GA"
3,2023,3,9E,Endeavor Air Inc.,Albany International,201.0,27.0,13.04,0.46,7.06,...,0.0,361.0,0.134328,0.002289,0.064876,0.035124,0.032040,0.034826,0.000000,"Albany, NY"
4,2023,3,9E,Endeavor Air Inc.,Hartsfield-Jackson Atlanta International,1598.0,222.0,57.22,8.08,61.32,...,0.0,6751.0,0.138924,0.005056,0.035807,0.038373,0.059687,0.005006,0.000000,"Atlanta, GA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21579,2022,3,YX,Republic Airline,Syracuse Hancock International,38.0,10.0,4.05,0.00,1.76,...,0.0,757.0,0.263158,0.000000,0.106579,0.046316,0.110263,0.026316,0.000000,"Syracuse, NY"
21580,2022,3,YX,Republic Airline,Tampa International,97.0,19.0,3.19,2.39,3.39,...,13.0,566.0,0.195876,0.024639,0.032887,0.034948,0.094433,0.041237,0.008969,"Tampa, FL"
21581,2022,3,YX,Republic Airline,Tulsa International,8.0,1.0,0.00,0.00,0.01,...,0.0,211.0,0.125000,0.000000,0.000000,0.001250,0.123750,0.000000,0.000000,"Tulsa, OK"
21582,2022,3,YX,Republic Airline,Eglin AFB Destin Fort Walton Beach,25.0,8.0,2.28,0.00,4.66,...,0.0,65.0,0.320000,0.000000,0.091200,0.186400,0.042400,0.080000,0.000000,"Valparaiso, FL"


In [13]:
df.dtypes

year_                            int64
month_                           int64
carrier                         object
carrier_name                    object
airport                         object
arr_flights                    float64
arr_del15                      float64
carrier_ct                     float64
weather_ct                     float64
nas_ct                         float64
security_ct                    float64
late_aircraft_ct               float64
arr_cancelled                  float64
arr_diverted                   float64
arr_delay                      float64
carrier_delay                  float64
weather_delay                  float64
nas_delay                      float64
security_delay                 float64
late_aircraft_delay            float64
arr_del15_per                  float64
arr_weather_delay_per          float64
arr_carrier_delay_per          float64
arr_nas_delay_per              float64
arr_late_aircraft_delay_per    float64
arr_cancelled_per        

In [14]:
# Delete nulls
df.dropna(how='any', inplace=True)

In [15]:
df.to_csv('AirlineDelays.csv', encoding='utf-8', index=False)

In [16]:
connection_string = "postgres:****************@localhost:5432/Travel"
engine = create_engine(f'postgresql://{connection_string}')

engine.table_names()


C:\Users\cassie.contreras\AppData\Local\Temp\ipykernel_22268\1148793964.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['airlinedelays', 'AirlineDelays']

In [17]:
# Load dataframe into database
df.to_sql(name='airlinedelays', con=engine, if_exists='append', index=False)

pd.read_sql_query('select * from airlinedelays', con=engine).head()



,year_,month_,carrier,carrier_name,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,...,security_delay,late_aircraft_delay,arr_del15_per,arr_weather_delay_per,arr_carrier_delay_per,arr_nas_delay_per,arr_late_aircraft_delay_per,arr_cancelled_per,arr_security_cancelled_per,location
0,2023,3,9E,Endeavor Air Inc.,Southwest Georgia Regional,89.0,8.0,4.46,1.00,1.61,...,0.0,59.0,0.089888,0.011236,0.050112,0.018090,0.010449,0.011236,0.0,"Albany, GA"
1,2023,3,9E,Endeavor Air Inc.,Alexandria International,62.0,8.0,3.95,0.37,1.29,...,0.0,118.0,0.129032,0.005968,0.063710,0.020806,0.038710,0.000000,0.0,"Alexandria, LA"
2,2023,3,9E,Endeavor Air Inc.,Augusta Regional at Bush Field,11.0,2.0,1.00,0.00,0.00,...,0.0,36.0,0.181818,0.000000,0.090909,0.000000,0.090909,0.000000,0.0,"Augusta, GA"
3,2023,3,9E,Endeavor Air Inc.,Albany International,201.0,27.0,13.04,0.46,7.06,...,0.0,361.0,0.134328,0.002289,0.064876,0.035124,0.032040,0.034826,0.0,"Albany, NY"
4,2023,3,9E,Endeavor Air Inc.,Hartsfield-Jackson Atlanta International,1598.0,222.0,57.22,8.08,61.32,...,0.0,6751.0,0.138924,0.005056,0.035807,0.038373,0.059687,0.005006,0.0,"Atlanta, GA"
